<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/moduler_optimalisering_evaulering/Moduler_optimalisering_evaluering_semiveiledet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotek

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
import time

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection
from statistics import mean, stdev

from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg


import optuna
from optuna.visualization import plot_optimization_history

# Egendefinerte Moduler

In [ ]:
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag
from moduler_trening_validering_testing_evaluering import umarkert_andel_dag


from moduler_trening_validering_testing_evaluering import evaluering_beregning
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval

Evalueringsmetrikker

In [ ]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

Andre moduler

In [ ]:
def bhd_andel_maks(data, antall_maks=100000):

  """
  Funksjon som finner maks andel til modell BHD grunnet begrensningn i RAM

  input
  - umarkert_data: datasett
  - antall_maks: maks antall data, 100 000

  output
  - andel_maks: andel av datasett
  """
  observasjoner = data.shape[0]
  forklaringsvariabler = data.shape[1]

  data_antall = observasjoner * forklaringsvariabler

  andel_maks = antall_maks/data_antall

  antall_maks = int(math.floor(antall_maks/forklaringsvariabler))

  if andel_maks >= 1.0:
    andel_maks = 1.0

  return andel_maks, antall_maks

In [ ]:
def andel_umarkert(andel, markert_data, umarkert_data):
  """
  Funksjon som finner andel umarkert data som tilsvarer gitt andel av markert

  input
  - andel: andel av markert data
  - markert_data: markert datasett
  - umarkert_data: umarkert datasett

  output
  - umarkert_andel: andel av umarkert datasett
  """

  antall_markert = markert_data.shape[0]
  antall_umarkert_total = umarkert_data.shape[0]

  antall_umarkert = antall_markert*andel

  andel_umarkert = antall_umarkert/antall_umarkert_total

  if andel_umarkert >= 1.0:
    andel_umarkert = 1.0

  return andel_umarkert

In [ ]:
def bhd_beregn_k(X_trening,
                 X_trening_markert,
                 X_test,
                 n_splitt=1):
  """
  Funksjon som beregner minimum og maksimim antall naboer for KNN graf
  konstruert i BHD algoritmen.

  Argumenter:
    - X_trening:
    - X_trening_markert:
    - X_test:
    - n_splitt:

  Returverdier:
    - k_min:
    - k_maks:
  """

  n_trening = X_trening.shape[0]
  n_trening_markert = X_trening_markert.shape[0]
  n_test = X_test.shape[0]
  n_validering = math.floor(n_trening_markert * (1/n_splitt)) # Estimert Validering_fold størrelse

  # Regner maks antall naboer ift.RAM
  andel_maks, antall_maks = bhd_andel_maks(X_trening)

  if andel_maks < 1.0:
    k_min = (antall_maks/2)
    k_maks = antall_maks

  else:

    k_min = int(n_trening/2)

    if n_test < n_validering:
      k_maks = int(n_trening + (n_test - n_validering)) - 1
    else:
      k_maks = int(n_trening)

  return k_min, k_maks

# Algoritme: SSL Regression Heat Diffusion on graphs

In [ ]:
import numpy as np
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.pairwise import rbf_kernel
from scipy import sparse

class BHD:


  """
  Grafbasert BHD-algoritmen for semi-veiledet læring, med domene kunnskap.

  Args:
      - k (int): Antall naboer i KNN-graf.
      - graf_vektet_harmonic (bool) = True: Bestemmer om det skal vektet graf
      skal brukes for beregning av harmonic score
      - initiell respons (str) = "global mean": Strategi for initialisering av
      markeringer for umarkert data. Standard er globalt gjennomsnitt.
      Mulige valg er 'global_mean', 'domain_mean', 'domain_median'.
      - alpha: (float) = 1.0: Parameter for heat diffusion.
      - maks_iterasjoner_harmonic (int) = 100: Maksimalt antall iterasjoner for
      oppdatering av markeringsvektor ved beregning av harmonic score.
      - maks_iterasjoner_heat (int) = 100: Maksimalt antall iterasjoner for
      oppdatering av markeringsvektor ved heat diffusion.

  """



  def __init__(self,
               k,
               graf_vektet_harmonic = True,
               initiell_respons = 'global_mean',
               alpha=1.0,
               maks_iterasjoner_harmonic=30,
               maks_iterasjoner_heat=100
               ):

      self.k = k
      self.graf_vektet_harmonic = graf_vektet_harmonic
      self.initiell_respons = initiell_respons
      self.alpha = alpha
      self.maks_iterasjoner_harmonic = maks_iterasjoner_harmonic
      self.maks_iterasjoner_heat = maks_iterasjoner_heat


  def fit_predict(self,
                  rå_data,
                  treningssett_markert,
                  treningssett_umarkert,
                  testsett,
                  X_trening_markert,
                  X_trening_umarkert,
                  X_test):

    """
    Trener BHD-modellen og predikerer markeringer for testdata

    Args:
        - rå_data (pd.DataFrame): Hele datasettet, ubehandlet.
        - treningssett_markert (pd.DataFrame): Markert treningsdata.
        - treningssett_umarkert (pd.DataFrame): Umarkerte treningsdata.
        - testsett (pd.DataFrame): Testdata.
        - X_trening_markert (pd.DataFrame): Matrise med forklaringsvariabler for
        markert treningsdata.
        - X_trening_umarkert (pd.DataFrame): Matrise med forklaringsvariabler for
          umarkert treningsdata.
        - X_test (pd.DataFrame): Matrise med forklaringsvariabler for testdata.

    Returns:
        - y_test_pred (np.ndarray): Predikerte markeringer for testdata.
        - y_trening_pred (np.ndarray): Predikerte markeringer for markert
        treningsdata (type: np.ndarray).
    """

    # Samler data som skal inkluderes etter valgt andel umarkert data, skalert forklaringsvariabler
    X_data = pd.concat([X_trening_markert, X_trening_umarkert, X_test], axis=0)

    # Samler data som skal inkluderes etter valgt andel umarkert data, responsvariabler
    y_data = pd.concat([treningssett_markert.iloc[:, -1],
                        treningssett_umarkert.iloc[:, -1],
                        testsett.iloc[:, -1]], axis=0)

    # Data verdier
    X_verdi = X_data.values
    y_verdi = y_data.values

    # Finner indekser til markert, umarkert og test data
    indeks_trening_markert = np.where(y_data.index.isin(treningssett_markert.index))[0]
    indeks_trening_umarkert = np.where(y_data.index.isin(treningssett_umarkert.index))[0]
    indeks_test = np.where(y_data.index.isin(testsett.index))[0]


    # Konstruerer graf og vektet graf (distansebasert)
    graf = self._get_graph_matrix(X_verdi)
    graf_vektet = self._get_weighted_knn_graph(X_verdi)


    # Beregner harmonic score (med uvektet eller vektet graf)

    # Bruker vektet graf hvis flagget sant
    if self.graf_vektet_harmonic:

      # Harmonic score - med vektet graf
      harmonic_scores = self._get_harmonic_scores(graf_vektet,
                                                  indeks_trening_markert,
                                                  y_data)
    # Bruker uvektet graf hvsi flagget usant
    else:
      # Harmonic score - uten vektet graf
      harmonic_scores = self._get_harmonic_scores(graf,
                                                  indeks_trening_markert,
                                                  y_data)



    # Initierer markeringer for all data
    initial_state = self._initialize_state_vector(rå_data,
                                                  treningssett_markert,
                                                  treningssett_umarkert,
                                                  testsett,
                                                  y_data,
                                                  indeks_trening_markert,
                                                  indeks_trening_umarkert,
                                                  indeks_test)


    # Heat diffusion with boundry condition (vektet graf)
    state = self._heat_diffusion_boundry_condition(graf_vektet,
                                                   harmonic_scores,
                                                   initial_state)

    # Frigjør minne
    del graf, graf_vektet

    # Prediksjoner
    y_test_pred = np.array(state[indeks_test].reshape(-1))
    y_trening_pred = np.array(state[indeks_trening_markert].reshape(-1))

    return y_test_pred, y_trening_pred



  # Interne moduler

  def _get_graph_matrix(self, X):

    """
    Lager KNN-graf fra matrise med forklaringsvariabler

    Args:
        - X (np.ndarray) : Matrise med forklaringsvariabler.

    Returns:
        - KNN_graf (scipy.sparse.csr_matrix): KNN-graf.
    """

    # KNN graf
    KNN_graf = kneighbors_graph(X,
                                n_neighbors=self.k,
                                mode="connectivity",
                                include_self=True,
                                n_jobs=-1)

    return KNN_graf


  def _get_weighted_knn_graph(self, X):

    """
    Lager KNN-graf fmed rbf vekter

    Argumenter:
        - X (np.ndarray) : Matrise med forklaringsvariabler.

    Returns:
        - KNN_graf_vektet (scipy.sparse.csr_matrix): KNN-graf med rbf vekter.
    """

    # KNN graf
    KNN_graf_vektet = kneighbors_graph(X,
                                       n_neighbors=self.k,
                                       mode="distance",
                                       include_self=True,
                                       n_jobs=-1)

    # Beregner RBF vekter
    distanser = KNN_graf_vektet.data
    nonzero_indekser = np.where(distanser != 0)[0]

    n_egenskaper = X.shape[1]
    sigma = 1.0 / n_egenskaper
    gamma = 1.0 / (2 * sigma ** 2)
    rbf_verdier = np.exp(-gamma * distanser[nonzero_indekser]**2)


    # Oppdaterer KNN med RBF vekter
    KNN_graf_vektet.data[nonzero_indekser] = rbf_verdier

    return KNN_graf_vektet

  # Initierer state_vector med domenekunnskap om enzymtyper
  def _initialize_state_vector(self,
                               rå_data,
                               treningssett_markert,
                               treningssett_umarkert,
                               testsett,
                               y_data,
                               indeks_trening_markert,
                               indeks_trening_umarkert,
                               indeks_test):

    """
    Initierer state_vector med domenekunnskap om enzymtyper som benyttes for
    heat diffusion. Umarkert data får midlertidige markering basert på
    domenekunnskap.

    Args:
        - rå_data (pd.DataFrame): Hele datasettet, ubehandlet.
        - treningssett_markert (pd.DataFrame): Markert treningsdata.
        - treningssett_umarkert (pd.DataFrame): Umarkerte treningsdata.
        - testsett (pd.DataFrame): Testdata.
        - y_data (np.ndarray): Matrise med responsvariabler for hele datasettet.
        - indeks_trening_markert (np.ndarray): Indekser for markert data i datasett.
        - indeks_trening_umarkert (np.ndarray): Indekser til umarkert data i datasett.
        - indeks_test (np.ndarray): Indekser til testdata i datasett.

    Returns:
        - initial_state (np.ndarray): State_vector med markeringer for all data.
    """

    initial_state = np.zeros(y_data.shape[0])

    # Legger inn markert data i state vector
    initial_state[indeks_trening_markert] = y_data[indeks_trening_markert]

    # Imputerer trening_umarkert og test etter gjennomsnitt av enzymtype
    # observasjoenen tilhører

    if (self.initiell_respons == "domain_mean") or (self.initiell_respons == "domain_median"):

      # Gjennomsnitt eller medinan på enzymtype
      initial_state[indeks_trening_umarkert] = self._respons_enzymtype(rå_data,
                                                            treningssett_markert,
                                                            treningssett_umarkert)

      initial_state[indeks_test] = self._respons_enzymtype(rå_data,
                                                            treningssett_markert,
                                                            testsett)

    else:
      # Globalt gjennomsnitt - ingen domenekunnskap
      initial_state[indeks_test] = np.mean(y_data[indeks_trening_markert])
      initial_state[indeks_trening_umarkert] = np.mean(y_data[indeks_trening_markert])

    return initial_state


  def _get_harmonic_scores(self,
                           graf,
                           indeks_trening_markert,
                           y_data):

    """
    Kalkulerer harmonic scores basert på (vektet) KNN_graf og markering fra
    markert data.

    Args:
        - graf (scipy.sparse.csr_matrix): (vektet) KNN-graf.
        - indeks_trening_markert (np.ndarray): Indekser til markert data i datasett .
        - y_data (np.ndarray): Matrise med responsvariabler for hele datasettet .

    Returns:
        - harmonic_scores (np.ndarray): Vektor med harmonic scores for all data.
    """
    n = graf.shape[0]

    # Kalkulerer degrees
    degrees = graf.sum(axis=0).A1
    degrees[degrees == 0] += np.finfo(float).eps # unngå å dele på 0

    # Transition matrix
    D = sparse.diags((1.0/degrees), offsets=0)
    P = D.dot(graf)

    # Normalisert trasistion matrix
    P_rad_sum = P.sum(axis=1).A1
    P_sum_diagonal = sparse.diags((1.0/P_rad_sum), offsets=0)

    P_normalized = P_sum_diagonal.dot(P)

    # Ønsker kun del av transition matrix som tilhører umarkert rader
    # Må gjøre det manuelt, da konvertering til lilmatrix går utover RAM
    for i in indeks_trening_markert:
      P_normalized.data[P_normalized.indptr[i]:P_normalized.indptr[i+1]] = 0



    # Initierer markeringer til all data
    alle_verdier = np.zeros(n)
    alle_verdier[indeks_trening_markert] = y_data[indeks_trening_markert]

    # Nomalisering av harmonic scores (Eget) - GIR best resultat
    # state_vector = alle_verdier / alle_verdier.sum()
    state_vector = alle_verdier

    # Antall loops
    gjenværende_iter = self.maks_iterasjoner_harmonic

    # State som skal oppdateres og original state som skal legges til
    state = state_vector.copy()
    U = state_vector.copy()

    # Oppdateres state
    while gjenværende_iter > 0:
      state = P_normalized.dot(state) + U
      gjenværende_iter -= 1

    # Runder av til 6 desimaler
    state = np.round(state, decimals=6)

    # Frigjør minne
    del P, D, degrees, P_normalized, state_vector, U, alle_verdier

    return state


  def _heat_diffusion_boundry_condition(self,
                                        graf_vektet,
                                        harmonic_scores,
                                        initial_state):

    """
    Gjennomfører heat diffusion med boundry betingelse og gir en endelig state
    kombinerrt med harmonic score.

    Args:
        - graf_vektet (scipy.sparse.csr_matrix): Vektet KNN-graf.
        - harmonic_scores (np.ndarray): Vektor med harmonic scores for all data.
        - initial_state (np.ndarray): State_vector med markeringer for all data.

    Returns:
        - state (np.ndarray): Oppdatert state_vector med markeringer for all data.
    """

    n = graf_vektet.shape[0]

    # Identity matrix
    I = sparse.eye(n, dtype=np.float64)

    # Laplacian matrix
    L = sparse.csgraph.laplacian(graf_vektet, normed=True).tocsr()

    # Iterasjoner
    M_iterasjoner = self.maks_iterasjoner_heat

    # Konstant C
    C = initial_state - harmonic_scores
    C = np.round(C, decimals=6)

    # Ledd som oppdaterer state i løkken
    V = I + (-self.alpha / M_iterasjoner) * L

    # Initiell state
    state = C.copy()
    state = state.reshape(n, 1)

    # Heat diffusion - går igjennom loop og oppdaterer state
    for b in range(1, M_iterasjoner+1):
      state =  V.dot(state)

    # Legger til harmonic score
    state = harmonic_scores.reshape(n, 1) + state

    # Frigjør minne
    del L, I, C, V

    return state

  def _respons_enzymtype(self,
                         rå_data,
                         data_markert,
                         data_umarkert):

    """
    Imputerer responsvariabel i umarkert datasett med globalt gjennomsnitt,
    gjennomsnitt eller median av tilsvarende enzymtype i markert datasett.

    Args:
        - rå_data (pd.DataFrame): Hele datasettet.
        - data_markert (pd.DataFrame): Markert treningsdata.
        - data_umarkert (pd.DataFrame): Umarkerte treningsdata.

    Returns:
        - data_umarkert_respons (pd.DataFrame): Data_umarkert med imputert responsvariabel.
    """

    # Enzymtyper markert data
    enzym_data_markert = rå_data.merge(data_markert,
                                      left_index=True,
                                      right_index=True,
                                      how='inner',
                                      suffixes=('','_drop'))['EnzymeCode']

    respons_variabel = data_markert.columns[-1]

    if self.initiell_respons == "domain_mean":

      # Domene gjennomsnitt
      enzymtype_respons = data_markert.join(enzym_data_markert).groupby('EnzymeCode')[respons_variabel].mean()

    elif self.initiell_respons == "domain_median":
      # Domene median
      enzymtype_respons = data_markert.join(enzym_data_markert).groupby('EnzymeCode')[respons_variabel].median()


    enzym_data_umarkert = rå_data.merge(data_umarkert,
                                        left_index=True,
                                        right_index=True,
                                        how='inner',
                                        suffixes=('','_drop'))['EnzymeCode']

    data_umarkert_enzym = data_umarkert.join(enzym_data_umarkert)

    for indeks, rad in data_umarkert_enzym.iterrows():
      enzym_type = rad['EnzymeCode']
      if enzym_type in enzymtype_respons:
        data_umarkert_enzym.loc[indeks, respons_variabel] = enzymtype_respons[enzym_type]

    return data_umarkert_enzym[respons_variabel]

# Selvtrent RFR

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

class SelfTrainingRandomForestRegressor:

  """
  En "Self-Learning" Random Forest Regressor for semi-veiledet læring

  Args:
        - n_estimators: Antall beslutningstrær (type: int).
        Standardverdien er 100.
        - criterion: Kriterium for valg av splittepunkt i beslutningstrær
         (type: str). Standardverdien er "squared_error".
        - max_depth: Maksimal dybde for beslutningstrær (type: int).
        Standardverdien er None (ingen begrensning).
        - min_samples_leaf: Minimum antall datapunkter i et enkelt blad
        (type: int). Standardverdien er 2.
        - max_features: Antall funksjoner i betraktning ved splitting
        (type: float). Standardverdien er 1.0.
        - maks_iterasjoner: Maksimalt antall iterasjoner for selvtrening
        (type: int). Standardverdien er 10.
        - std_terskel: Terskel for relativ standardavvik for inkludering av
        psuedomarkert data som markere data (type: float).
        Standardverdien er 0.01.
        - random_state: Tilfeldighets seed (type: int). Standardverdien er 123.

    Attributes:
        - X_labeled: Matrise med markerte data (type: np.ndarray).
        - y_labeled: Vektor for responsvariabel med markerte data
        (type: np.ndarray).
        - X_unlabeled: Matrise med umarkerte data (type: np.ndarray).
        - model: RandomForestRegressor-modell
        (type: sklearn.ensemble.RandomForestRegressor).
  """

  def __init__(self,
                n_estimators=100,
                criterion="squared_error",
                max_depth = None,
                min_samples_leaf = 2,
                max_features = 1.0,
                maks_iterasjoner=10,
                std_terskel=0.01,
                random_state = 123):

      self.maks_iterasjoner = maks_iterasjoner
      self.std_terskel = std_terskel
      self.model = RandomForestRegressor(n_estimators=n_estimators,
                                         criterion=criterion,
                                         max_depth=max_depth,
                                         min_samples_leaf=min_samples_leaf,
                                         max_features=max_features,
                                         n_jobs=-1,
                                         random_state=random_state)

      self.X_labeled = None
      self.y_labeled = None
      self.X_unlabeled = None

  def fit(self, X_labeled, y_labeled, X_unlabeled):

    """
    Modelltrening ved bruk av "self-learning"

    Args:
        - X_labeled: Matrise med markerte data (type: np.ndarray).
        - y_labeled: Vektor for responsvariabel med markerte data
        (type: np.ndarray).
        - X_unlabeled: Matrise med umarkerte data (type: np.ndarray).

    """

    self.X_labeled = X_labeled
    self.y_labeled = y_labeled
    self.X_unlabeled = X_unlabeled

    for i in range(self.maks_iterasjoner):

        # Trener modell på midlertidig markert datasett
        self.model.fit(self.X_labeled, self.y_labeled)

        # Midlertidige markeringer med prediskjoner på umarkert data
        pseudo_labels = self.model.predict(self.X_unlabeled)

        # Samler individulle prediksjoner fra ensemble av trær
        prediksjoner = np.array([tree.predict(self.X_unlabeled) for tree in self.model.estimators_])

        # Beregner gjennomsnitt og standardavvik av prediksjoner
        snitt = np.mean(prediksjoner, axis=0)
        std = np.std(prediksjoner, axis=0)

        # Beregner relativ standardavvik
        usikkerhet = std/snitt

        # Lager maske for kriteriet
        mask = usikkerhet < self.std_terskel

        # Legger til de observasjoner fra umarkert datasett som oppfyller kravet om relativ standardavvik
        if mask.any():
            self.X_labeled = np.concatenate([self.X_labeled, self.X_unlabeled[mask]], axis=0)
            self.y_labeled = np.concatenate([self.y_labeled, pseudo_labels[mask]], axis=0)

              # Utelukker inkludert data fra sett med umarkert data
            self.X_unlabeled = self.X_unlabeled[~mask]

        # Bryter ut av loop dersom ingen obserasjoner oppfyller krav
        else:
            break

  def predict(self, X):

    """
    Predikerer markeringer for ny data

    Args:
        - X: Matrise med ny data (type: np.ndarray).

    Returns:
        - Vektor med predikerte markeringer (type: np.ndarray)
    """

    return self.model.predict(X)

# Moduler for optimalisering etter treningsandel etter algoritme

## Modelltrening etter enzymtype

### CoReg

In [ ]:
def trening_coreg_enzym(treningssett_markert,
                        enzymtyper_treningssett_markert,
                        treningssett_umarkert,
                        treningssett,
                        testsett,
                        n_splitt,
                        n_gjentagelser,
                        n_runder,
                        random_seed=2024):

  """
  Funksjonen for optimalisering og trening av CoReg-modell med
  "Alternativ 1: Fordelt på enzymtype" som oppdelingsmetode for
  trening, validering og testing for å forutsi responsvariabel
  fra datasett med markert og umarkert data.

  Argumenter:
    - treningssett_markert (pandas.DataFrame): Dataframe med markert data.
    - enzymtyper_treningssett_markert (np.array): Vektor med enzymtyper for
    markert data.
    - treningssett_umarkert (pandas.DataFrame): Dataframe med umarkert data.
    - treningssett (pandas.DataFrame): Dataframe med all data
    (markert og umarkert).

    - testsett (pd.DataFrame):
    - n_splitt (int): Antall ganger dataene deles inn i fold for validering
    (default: 5).
    - n_gjentagelser (int): Antall ganger valideringsprosessen skal gjentas for
    hver fold (default: 2).
    - trenings_andel (float): Andel av avsatt markert treningsdaata som benyttes
    til trening (default: 1.0).
    - random_seed (int): Tilfeldig frø (default: 123).

  Returnerer:
      - resultater (list): Liste med evalueringsmålinger
      (rmse_test, r2_test, mae_test, mape_test,
      rmse_trening, r2_trening, mae_trening, mape_trening,
      trening_markert_antall, tid_trening, n_runder, splitt, tid_optimalisering).
      - beste_parametere (dict): Dictionary med optimale hyperparameterne for
      modellen.
      - umarkert_antall_benyttet (int): Antall umarkerte data benyttet av modellen.
  """

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]


  # Splitter data i forklaring og responsvariabler
  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # Skalerer data
  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  #CoReg
  def objective(trial):

    parametere = {
        'k1': trial.suggest_int('k1', 1, 10),
        'k2': trial.suggest_int('k2', 1, 10),
        'p1': trial.suggest_int('p1', 1, 10),
        'p2': trial.suggest_int('p2', 1, 10),
        'max_iters': trial.suggest_int('max_iters', 1, 50),
        'pool_size': trial.suggest_int('pool_size', 2, 100)
    }

    # Setter opp modell
    coreg = CoReg(**parametere)

    # Gjør klar til kryssvalidering
    rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                   n_repeats=n_gjentagelser,
                                   random_state=random_seed)

    # Lagring av resultater ved kryssvalidering
    rmse_validering_resultat = []

    step = 0
    # Gør igjennom kryssvalidering
    for train, val in rskf.split(treningssett_markert,
                                 enzymtyper_treningssett_markert):

      # Markerte forklaringsvariabler folds
      trenings_fold_markert = treningssett_markert.iloc[train, :]
      val_fold = treningssett_markert.iloc[val, :]

      # Markert respons folds
      y_trening_fold = trenings_fold_markert.iloc[:, -1]
      y_val_fold = val_fold.iloc[:, -1]


      #Fullstendig treningsdata, markert og umarkert folds
      X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
      X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert], axis=0)

      # Skalerer folds etter treningssett fold, inkl. markert og umarkert
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_markert_transformert = skalerer_fold.transform(X_trening_fold_markert)
      X_trening_fold_umarkert_transformert = skalerer_fold.transform(X_trening_umarkert)

      X_val_fold = val_fold.iloc[:, :-1]
      X_val_fold_transformert = skalerer_fold.transform(X_val_fold)

      # Trener modell og predikerer på valideringsdata
      coreg.fit(X_trening_fold_markert_transformert,
                y_trening_fold,
                X_trening_fold_umarkert_transformert)

      y_val_pred = coreg.predict(X_val_fold_transformert)

      # Evaluerer modell
      rmse_validering = rmse(y_val_fold, y_val_pred)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step > math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modeloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=15,
                                  reduction_factor=2))
  study.optimize(objective, n_trials=n_runder, n_jobs=-1)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  coreg = CoReg(**beste_parametere)

  # Trener modell for videre evaluering
  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  coreg.fit(X_trening_markert_transformert,
            y_trening_markert.values,
            X_trening_umarkert_transformert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Prediksjoner
  y_test_pred = coreg.predict(X_test_transformert)
  y_trening_pred = coreg.predict(X_trening_markert_transformert)


  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)



  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  # Umarkert benyttet
  umarkert_antall_benyttet = X_trening_umarkert.shape[0]

  return resultater, beste_parametere, rmse_score_opt, umarkert_antall_benyttet

###Selvtrent-RFR

In [ ]:
def trening_selvtrent_rfr_enzym(treningssett_markert,
                                enzymtyper_treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                testsett,
                                n_splitt,
                                n_gjentagelser,
                                n_runder,
                                random_seed=2024):

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Splitter data i forklaring og responsvariabler
  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]


  # Skalerer data
  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  # Selvtrent-RFR
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 500),
          'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']),
          'max_depth': trial.suggest_int('max_depth', 1, 30),
          'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
          'max_features': trial.suggest_float('max_features', 0.1, 1.0),
          'maks_iterasjoner': trial.suggest_int('maks_iterasjoner', 1, 20),
          'std_terskel': trial.suggest_float('std_terskel', 0.01, 0.05)
      }

    selv_rfr = SelfTrainingRandomForestRegressor(**parametere)

    # Kryssvalidering
    rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                   n_repeats=n_gjentagelser,
                                   random_state=random_seed)

    # Lagring av resultat
    rmse_validering_resultat = []

    step = 0
    for train, val in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      # Markerte folds
      trenings_fold_markert = treningssett_markert.iloc[train, :]
      val_fold = treningssett_markert.iloc[val, :]

      # Markert respons
      y_trening_fold = trenings_fold_markert.iloc[:, -1]
      y_val_fold = val_fold.iloc[:, -1]


      #Fullstendig treningsdata, markert og umarkert folds
      X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
      X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert],
                                 axis=0)

      # Skalerer folds etter treningsset fold, inkl. markert og umarkert
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_markert_transformert = skalerer_fold.transform(X_trening_fold_markert)
      X_trening_fold_umarkert_transformert = skalerer_fold.transform(X_trening_umarkert)

      X_val_fold = val_fold.iloc[:, :-1]
      X_val_fold_transformert = skalerer_fold.transform(X_val_fold)

      selv_rfr.fit(X_trening_fold_markert_transformert,
                   y_trening_fold,
                   X_trening_fold_umarkert_transformert)

      y_val_pred = selv_rfr.predict(X_val_fold_transformert)
      rmse_validering = rmse(y_val_fold, y_val_pred)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step > math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modelloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=15,
                                  reduction_factor=2))
  study.optimize(objective, n_trials=n_runder, n_jobs=-1)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  selv_rfr = SelfTrainingRandomForestRegressor(**beste_parametere)

  # Trener modell for videre evaluering

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  selv_rfr.fit(X_trening_markert_transformert,
               y_trening_markert.values,
               X_trening_umarkert_transformert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Prediksjoner
  y_test_pred = selv_rfr.predict(X_test_transformert)
  y_trening_pred = selv_rfr.predict(X_trening_markert_transformert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  # Umarkert benyttet
  umarkert_antall_benyttet = X_trening_umarkert.shape[0] - selv_rfr.X_unlabeled.shape[0]

  return resultater, beste_parametere, rmse_score_opt, umarkert_antall_benyttet

### BHD domene

In [ ]:
def trening_bhd_domene_enzym(rå_data,
                              treningssett_markert,
                              enzymtyper_treningssett_markert,
                              treningssett_umarkert,
                              enzymtyper_treningssett_umarkert,
                              treningssett,
                              testsett,
                              n_splitt,
                              n_gjentagelser,
                              n_runder,
                              random_seed=2024):


  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Test data
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # Markert data
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # Umarkert data
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]

  # Trenigsdata, markert og umarkert
  X_trening = pd.concat([X_trening_markert, X_trening_umarkert], axis=0)
  X_trening = X_trening.sort_index(ascending=True)

  # Skalerer data etter treningsdata, inkl. markert og umarkert
  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = X_trening_markert.copy()
  X_trening_markert_transformert.iloc[:, :] = skalerer.transform(X_trening_markert)

  X_trening_umarkert_transformert = X_trening_umarkert.copy()
  X_trening_umarkert_transformert.iloc[:, :] = skalerer.transform(X_trening_umarkert)

  X_test_transformert = X_test.copy()
  X_test_transformert.iloc[:,:] = skalerer.transform(X_test)

  # Parameter tuning
  def objective(trial):

    # Intervall for KNN-graf basert størrelse på treningsdata
    # Finner intervall for k
    k_min, k_maks = bhd_beregn_k(X_trening,
                                 X_trening_markert_transformert,
                                 X_test_transformert,
                                 n_splitt)


    parametere = {
          'k': trial.suggest_int('k', k_min, k_maks),
          'graf_vektet_harmonic': trial.suggest_categorical('graf_vektet_harmonic', [False, True]),
          'initiell_respons': trial.suggest_categorical('initiell_respons', ['domain_mean', 'domain_median']),
          'alpha': trial.suggest_float('alpha', 0.0, 1.0, step=0.001),
          'maks_iterasjoner_harmonic': trial.suggest_int('maks_iterasjoner_harmonic', 1, 100),
          'maks_iterasjoner_heat': trial.suggest_int('maks_iterasjoner_heat', 1, 100)
      }

    bhd = BHD(**parametere)

    # Kryss-validering
    rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                   n_repeats=n_gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, val in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      # Markerte folds
      trenings_fold_markert = treningssett_markert.iloc[train, :]
      val_fold = treningssett_markert.iloc[val, :]

      # Markert respons
      y_trening_fold = trenings_fold_markert.iloc[:, -1]
      y_val_fold = val_fold.iloc[:, -1]

      #Fullstendig treningsdata, markert og umarkert folds
      X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
      X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert], axis=0)

      #Skalerer folds etter treningssett fold, inkl. markert og umarkert
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_markert_transformert = X_trening_fold_markert.copy()
      X_trening_fold_markert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_fold_markert)

      X_trening_fold_umarkert_transformert = X_trening_umarkert.copy()
      X_trening_fold_umarkert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_umarkert)

      X_val_fold = val_fold.iloc[:, :-1]
      X_val_fold_transformert = X_val_fold.copy()
      X_val_fold_transformert.iloc[:, :] = skalerer_fold.transform(X_val_fold)

      y_val_pred, _ = \
      bhd.fit_predict(rå_data,
                      trenings_fold_markert,
                      treningssett_umarkert,
                      val_fold,
                      X_trening_fold_markert_transformert,
                      X_trening_fold_umarkert_transformert,
                      X_val_fold_transformert)

      rmse_validering = rmse(y_val_fold, y_val_pred)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step > math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1



    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modelloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=15,
                                  reduction_factor=2))
  study.optimize(objective, n_trials=n_runder, n_jobs=-1)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  bhd = BHD(**beste_parametere)

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  # Prediksjoner
  y_test_pred, y_trening_pred\
  = bhd.fit_predict(rå_data,
                    treningssett_markert,
                    treningssett_umarkert,
                    testsett,
                    X_trening_markert_transformert,
                    X_trening_umarkert_transformert,
                    X_test_transformert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))



  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  # Umarkert benyttet
  umarkert_antall_benyttet = X_trening_umarkert.shape[0]

  return resultater, beste_parametere, rmse_score_opt, umarkert_antall_benyttet

## Modelltrening etter dag og kontinuitet

### CoReg

In [ ]:
def trening_coreg_dag(rå_data,
                      treningssett_markert,
                      treningssett_umarkert,
                      treningssett,
                      testsett,
                      n_splitt,
                      n_runder,
                      random_seed=2024):



  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Deler data
  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # Skalering
  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  def objective(trial):

    parametere = {
        'k1': trial.suggest_int('k1', 1, 20),
        'k2': trial.suggest_int('k2', 1, 20),
        'p1': trial.suggest_int('p1', 1, 10),
        'p2': trial.suggest_int('p2', 1, 10),
        'max_iters': trial.suggest_int('max_iters', 1, 50),
        'pool_size': trial.suggest_int('pool_size', 2, 200)
    }


    coreg = CoReg(**parametere)

    # Kryssvalidering
    treningssett, val_sett\
    = trening_validering_oppdeling_dag(rå_data,
                                       treningssett_markert,
                                       n_splitt=n_splitt,
                                       test_ekskluder_kalkun_kylling=True)
    rmse_validering_resultat = []

    step = 0
    for i, j in zip(treningssett, val_sett):
      train = treningssett[i]
      val = val_sett[j]

      # Markerte forklaringsvariabler folds
      trenings_fold_markert = treningssett_markert.loc[train, :]
      val_fold = treningssett_markert.loc[val, :]

      # Markert respons folds
      y_trening_fold = trenings_fold_markert.iloc[:, -1]
      y_val_fold = val_fold.iloc[:, -1]


      #Fullstendig treningsdata, markert og umarkert folds
      X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
      X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert], axis=0)

      # Skalerer folds etter treningssett fold, inkl. markert og umarkert
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_markert_transformert = skalerer_fold.transform(X_trening_fold_markert)
      X_trening_fold_umarkert_transformert = skalerer_fold.transform(X_trening_umarkert)

      X_val_fold = val_fold.iloc[:, :-1]
      X_val_fold_transformert = skalerer_fold.transform(X_val_fold)

      # Trener modell og predikerer på valideringsdata
      coreg.fit(X_trening_fold_markert_transformert,
                y_trening_fold,
                X_trening_fold_umarkert_transformert)

      y_val_pred = coreg.predict(X_val_fold_transformert)

      # Evaluerer modell
      rmse_validering = rmse(y_val_fold, y_val_pred)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step > math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modeloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=15,
                                  reduction_factor=2))
  study.optimize(objective, n_trials=n_runder, n_jobs=-1)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  coreg = CoReg(**beste_parametere)

  # Trener modell for videre evaluering
  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  coreg.fit(X_trening_markert_transformert,
            y_trening_markert.values,
            X_trening_umarkert_transformert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Prediksjoner
  y_test_pred = coreg.predict(X_test_transformert)
  y_trening_pred = coreg.predict(X_trening_markert_transformert)


  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)



  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  # Umarkert benyttet
  umarkert_antall_benyttet = X_trening_umarkert.shape[0]

  return resultater, beste_parametere, rmse_score_opt, umarkert_antall_benyttet

### Selvtrent RFR

In [ ]:
def trening_selvtrent_rfr_dag(rå_data,
                              treningssett_markert,
                              treningssett_umarkert,
                              treningssett,
                              testsett,
                              n_splitt,
                              n_runder,
                              random_seed=2024):

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Deler opp datasett i forklaring og responsvariabler
  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]


  # Skalerer data
  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  # Selvtrent-RFR
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 500),
          'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']),
          'max_depth': trial.suggest_int('max_depth', 1, 30),
          'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
          'max_features': trial.suggest_float('max_features', 0.1, 1.0),
          'maks_iterasjoner': trial.suggest_int('maks_iterasjoner', 1, 20),
          'std_terskel': trial.suggest_float('std_terskel', 0.01, 0.05)
      }

    selv_rfr = SelfTrainingRandomForestRegressor(**parametere)

    # Kryssvalidering
    treningssett, val_sett\
     = trening_validering_oppdeling_dag(rå_data,
                                        treningssett_markert,
                                        n_splitt=n_splitt,
                                        test_ekskluder_kalkun_kylling=True)
    rmse_validering_resultat = []

    step = 0
    for i, j in zip(treningssett, val_sett):
      train = treningssett[i]
      val = val_sett[j]

       # Markerte folds
      trenings_fold_markert = treningssett_markert.loc[train, :]
      val_fold = treningssett_markert.loc[val, :]

      # Markert respons
      y_trening_fold = trenings_fold_markert.iloc[:, -1]
      y_val_fold = val_fold.iloc[:, -1]


      #Fullstendig treningsdata, markert og umarkert folds
      X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
      X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert],
                                 axis=0)

      # Skalerer folds etter treningsset fold, inkl. markert og umarkert
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_markert_transformert = skalerer_fold.transform(X_trening_fold_markert)
      X_trening_fold_umarkert_transformert = skalerer_fold.transform(X_trening_umarkert)

      X_val_fold = val_fold.iloc[:, :-1]
      X_val_fold_transformert = skalerer_fold.transform(X_val_fold)

      selv_rfr.fit(X_trening_fold_markert_transformert,
                   y_trening_fold,
                   X_trening_fold_umarkert_transformert)

      y_val_pred = selv_rfr.predict(X_val_fold_transformert)
      rmse_validering = rmse(y_val_fold, y_val_pred)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step > math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modelloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=15,
                                  reduction_factor=2))
  study.optimize(objective, n_trials=n_runder, n_jobs=-1)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  selv_rfr = SelfTrainingRandomForestRegressor(**beste_parametere)

  # Trener modell for videre evaluering

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  selv_rfr.fit(X_trening_markert_transformert,
               y_trening_markert.values,
               X_trening_umarkert_transformert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Prediksjoner
  y_test_pred = selv_rfr.predict(X_test_transformert)
  y_trening_pred = selv_rfr.predict(X_trening_markert_transformert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  # Umarkert benyttet
  umarkert_antall_benyttet = X_trening_umarkert.shape[0] - selv_rfr.X_unlabeled.shape[0]

  return resultater, beste_parametere, rmse_score_opt, umarkert_antall_benyttet

### BHD - Domene

In [ ]:
def trening_bhd_domene_dag(rå_data,
                            treningssett_markert,
                            treningssett_umarkert,
                            testsett,
                            n_splitt,
                            n_runder,
                            random_seed=2024):

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Test data
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # Markert data
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # Umarkert data
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]


  # Trenigsdata, markert og umarkert
  X_trening = pd.concat([X_trening_markert, X_trening_umarkert], axis=0)

  # Skalerer data etter treningsdata, inkl. markert og umarkert
  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = X_trening_markert.copy()
  X_trening_markert_transformert.iloc[:, :] = skalerer.transform(X_trening_markert)

  X_trening_umarkert_transformert = X_trening_umarkert.copy()
  X_trening_umarkert_transformert.iloc[:, :] = skalerer.transform(X_trening_umarkert)

  X_test_transformert = X_test.copy()
  X_test_transformert.iloc[:,:] = skalerer.transform(X_test)

  # Parameter tuning
  def objective(trial):

    # Intervall for KNN-graf basert størrelse på treningsdata
    # Finner intervall for k
    k_min, k_maks = bhd_beregn_k(X_trening,
                                 X_trening_markert_transformert,
                                 X_test_transformert,
                                 n_splitt)

    parametere = {
          'k': trial.suggest_int('k', k_min, k_maks),
          'graf_vektet_harmonic': trial.suggest_categorical('graf_vektet_harmonic', [False, True]),
          'initiell_respons': trial.suggest_categorical('initiell_respons', ['domain_mean', 'domain_median', 'global_mean']),
          'alpha': trial.suggest_float('alpha', 0, 1.0),
          'maks_iterasjoner_harmonic': trial.suggest_int('maks_iterasjoner_harmonic', 1, 100),
          'maks_iterasjoner_heat': trial.suggest_int('maks_iterasjoner_heat', 1, 100)
      }

    bhd = BHD(**parametere)

    # Kryss-validering

    treningssett, val_sett\
    = trening_validering_oppdeling_dag(rå_data,
                                       treningssett_markert,
                                       n_splitt=n_splitt,
                                       test_ekskluder_kalkun_kylling=True)
    rmse_validering_resultat = []

    step = 0
    for i, j in zip(treningssett, val_sett):
      train = treningssett[i]
      val = val_sett[j]

      # Markerte folds
      trenings_fold_markert = treningssett_markert.loc[train, :]
      val_fold = treningssett_markert.loc[val, :]

      # Markert respons
      y_trening_fold = trenings_fold_markert.iloc[:, -1]
      y_val_fold = val_fold.iloc[:, -1]

      #Fullstendig treningsdata, markert og umarkert folds
      X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
      X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert], axis=0)

      #Skalerer folds etter treningssett fold, inkl. markert og umarkert
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_markert_transformert = X_trening_fold_markert.copy()
      X_trening_fold_markert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_fold_markert)

      X_trening_fold_umarkert_transformert = X_trening_umarkert.copy()
      X_trening_fold_umarkert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_umarkert)

      X_val_fold = val_fold.iloc[:, :-1]
      X_val_fold_transformert = X_val_fold.copy()
      X_val_fold_transformert.iloc[:, :] = skalerer_fold.transform(X_val_fold)

      y_val_pred, _ = \
      bhd.fit_predict(rå_data,
                      trenings_fold_markert,
                      treningssett_umarkert,
                      val_fold,
                      X_trening_fold_markert_transformert,
                      X_trening_fold_umarkert_transformert,
                      X_val_fold_transformert)

      rmse_validering = rmse(y_val_fold, y_val_pred)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step > math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modelloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=15,
                                  reduction_factor=2))
  study.optimize(objective, n_trials=n_runder, n_jobs=-1)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  bhd = BHD(**beste_parametere)

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  # Prediksjoner
  y_test_pred, y_trening_pred\
  = bhd.fit_predict(rå_data,
                    treningssett_markert,
                    treningssett_umarkert,
                    testsett,
                    X_trening_markert_transformert,
                    X_trening_umarkert_transformert,
                    X_test_transformert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))



  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  # Umarkert benyttet
  umarkert_antall_benyttet = X_trening_umarkert.shape[0]

  return resultater, beste_parametere, rmse_score_opt, umarkert_antall_benyttet

# Modellevaluering etter enzymtype

In [ ]:
def semi_modell_evaluering_1(dict_res_opt: dict,
                             dict_res: dict,
                             rå_data: pd.DataFrame,
                             treningssett_markert: pd.DataFrame,
                             enzymtyper_treningssett_markert: pd.DataFrame,
                             treningssett_umarkert: pd.DataFrame,
                             enzymtyper_treningssett_umarkert: pd.DataFrame,
                             treningssett: pd.DataFrame,
                             testsett: pd.DataFrame,
                             n_splitt: int,
                             n_gjentagelser: int,
                             n_runder: int,
                             test_andel: float,
                             trenings_andel: float,
                             random_seed: int):

  """
  Funksjon for evaluering av semiveiledet modeller "Coreg", "BHD" og


  input:
  -
  - modell:

  output:

  """

  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Umarkert
  umarkert_antall = treningssett_umarkert.shape[0]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]

  # Modelloptimalisering - CoReg
  coreg_res, coreg_param, coreg_opt_rmse, coreg_umarkert_antall_benyttet\
  = trening_coreg_enzym(treningssett_markert,
                        enzymtyper_treningssett_markert,
                        treningssett_umarkert,
                        treningssett,
                        testsett,
                        n_splitt,
                        n_gjentagelser,
                        n_runder,
                        random_seed=random_seed)

  # Lagring av informasjon om paramtere for treningsandel
  res_eval_opt(dict_res_opt, 'CoReg', test_andel,
               trenings_andel, coreg_param, coreg_opt_rmse, coreg_res,
               umarkert_antall, coreg_umarkert_antall_benyttet)

  # Coreg - beste modell
  # Initierer modell
  coreg = CoReg(**coreg_param)

    # Modelloptimalisering - Selv_RFR
  selv_rfr_res, selv_rfr_param, selv_rfr_opt_rmse, selv_rfr_umarkert_antall_benyttet\
  = trening_selvtrent_rfr_enzym(treningssett_markert,
                                enzymtyper_treningssett_markert,
                                treningssett_umarkert,
                                treningssett,
                                testsett,
                                n_splitt,
                                n_gjentagelser,
                                n_runder,
                                random_seed=random_seed)

  # Lagring av informasjon om paramtere for treningsandel
  res_eval_opt(dict_res_opt, 'Selv_RFR', test_andel,
               trenings_andel, selv_rfr_param, selv_rfr_opt_rmse, selv_rfr_res,
               umarkert_antall, selv_rfr_umarkert_antall_benyttet)

  # Selv_RFR - beste modell
  # Initierer modell
  selv_rfr = SelfTrainingRandomForestRegressor(**selv_rfr_param)


  # Modelloptimalisering - BHD
  bhd_domene_res, bhd_domene_param, bhd_domene_opt_rmse, bhd_domene_umarkert_antall_benyttet\
  = trening_bhd_domene_enzym(rå_data,
                              treningssett_markert,
                              enzymtyper_treningssett_markert,
                              treningssett_umarkert,
                              enzymtyper_treningssett_umarkert,
                              treningssett,
                              testsett,
                              n_splitt,
                              n_gjentagelser,
                              n_runder,
                              random_seed=random_seed)


  # Lagring av informasjon om paramtere for treningsandel
  res_eval_opt(dict_res_opt, 'BHD - Domene', test_andel,
               trenings_andel, bhd_domene_param, bhd_domene_opt_rmse, bhd_domene_res,
               umarkert_antall, bhd_domene_umarkert_antall_benyttet)

  bhd_domene = BHD(**bhd_domene_param)


  # Lagring av resultater fra kryssvalidering
  coreg_res = Resultat_evaluering()
  selv_rfr_res = Resultat_evaluering()
  bhd_domene_res = Resultat_evaluering()

  # Kryssvalidering
  rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                 n_repeats=n_gjentagelser,
                                 random_state=random_seed)


  # Går igjennom kryssvalidering
  for train, val in rskf.split(treningssett_markert,
                               enzymtyper_treningssett_markert):

      # Markerte folds
      trenings_fold_markert = treningssett_markert.iloc[train, :]
      val_fold = treningssett_markert.iloc[val, :]

      # Markert respons
      y_trening_fold = trenings_fold_markert.iloc[:, -1]
      y_val_fold = val_fold.iloc[:, -1]


      #Fullstendig treningsdata, markert og umarkert folds
      X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
      X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert],
                                 axis=0)

      # Skalerer folds etter treningsset fold, inkl. markert og umarkert
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      # Markert
      X_trening_fold_markert_transformert = X_trening_fold_markert.copy()
      X_trening_fold_markert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_fold_markert)

      # Umarkert
      X_trening_fold_umarkert_transformert = X_trening_umarkert.copy()
      X_trening_fold_umarkert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_umarkert)

      # Validering
      X_val_fold = val_fold.iloc[:, :-1]
      X_val_fold_transformert = X_val_fold.copy()
      X_val_fold_transformert.iloc[:, :] = skalerer_fold.transform(X_val_fold)

      # Test
      X_test_transformert = X_test.copy()
      X_test_transformert.iloc[:, :] = skalerer_fold.transform(X_test)


      # CoReg
      coreg.fit(X_trening_fold_markert_transformert.values,
                y_trening_fold.values,
                X_trening_fold_umarkert_transformert.values)

      #Prediksjoner
      y_trening_pred = coreg.predict(X_trening_fold_markert_transformert.values)
      y_val_pred = coreg.predict(X_val_fold_transformert.values)
      y_test_pred = coreg.predict(X_test_transformert.values)

      # Evaluering
      coreg_res.evaluering(y_trening_fold,
                           y_trening_pred,
                           y_val_fold,
                           y_val_pred,
                           y_test,
                           y_test_pred)

      #Coreg umarkert benyttet
      coreg_umarkert_antall_benyttet = X_trening_umarkert.shape[0]

      # Selv_RFR
      selv_rfr.fit(X_trening_fold_markert_transformert.values,
                   y_trening_fold.values,
                   X_trening_fold_umarkert_transformert.values)


      # Prediksjoner
      y_trening_pred = selv_rfr.predict(X_trening_fold_markert_transformert.values)
      y_val_pred = selv_rfr.predict(X_val_fold_transformert.values)
      y_test_pred = selv_rfr.predict(X_test_transformert.values)

      # Evaluering
      selv_rfr_res.evaluering(y_trening_fold,
                              y_trening_pred,
                              y_val_fold,
                              y_val_pred,
                              y_test,
                              y_test_pred)

      #Selv_RFR umarkert benyttet
      selv_rfr_umarkert_antall_benyttet = X_trening_umarkert.shape[0] - selv_rfr.X_unlabeled.shape[0]


      # BHD - Domene

      # Prediksjoner
      y_val_pred, y_trening_pred\
      = bhd_domene.fit_predict(rå_data,
                              trenings_fold_markert,
                              treningssett_umarkert,
                              val_fold,
                              X_trening_fold_markert_transformert,
                              X_trening_fold_umarkert_transformert,
                              X_val_fold_transformert)

      y_test_pred, y_trening_pred\
      = bhd_domene.fit_predict(rå_data,
                              trenings_fold_markert,
                              treningssett_umarkert,
                              testsett,
                              X_trening_fold_markert_transformert,
                              X_trening_fold_umarkert_transformert,
                              X_test_transformert)

      # Evaluering
      bhd_domene_res.evaluering(y_trening_fold,
                                y_trening_pred,
                                y_val_fold,
                                y_val_pred,
                                y_test,
                                y_test_pred)

  # Lagring av informasjon om paramtere for

  res_eval(dict_res, 'Coreg', test_andel,
           trenings_andel, trening_markert_antall,
           parametere=coreg_param,
           rmse_score_opt=coreg_opt_rmse,
           n_splitt=n_splitt,
           res_eval=coreg_res,
           umarkert_antall=umarkert_antall,
           umarkert_benyttet=coreg_umarkert_antall_benyttet)


  res_eval(dict_res, 'Selv_RFR', test_andel,
           trenings_andel, trening_markert_antall,
           parametere=selv_rfr_param,
           rmse_score_opt=selv_rfr_opt_rmse,
           n_splitt=n_splitt,
           res_eval=selv_rfr_res,
           umarkert_antall=umarkert_antall,
           umarkert_benyttet=selv_rfr_umarkert_antall_benyttet)

  res_eval(dict_res, 'BHD - Domene', test_andel,
           trenings_andel, trening_markert_antall,
           parametere=bhd_domene_param,
           rmse_score_opt=bhd_domene_opt_rmse,
           n_splitt=n_splitt,
           res_eval=bhd_domene_res,
           umarkert_antall=umarkert_antall,
           umarkert_benyttet=bhd_domene_umarkert_antall_benyttet)

# Modelevaluering etter dag og kontinuitet

In [ ]:
def semi_modell_evaluering_2(dict_res_opt: dict,
                             dict_res: dict,
                             rå_data: pd.DataFrame,
                             treningssett_markert: pd.DataFrame,
                             treningssett_umarkert: pd.DataFrame,
                             treningssett: pd.DataFrame,
                             testsett: pd.DataFrame,
                             n_splitt: int,
                             n_runder: int,
                             test_andel: float,
                             trenings_andel: float,
                             random_seed: int):

  """

  """


  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Umarkert antall
  umarkert_antall = treningssett_umarkert.shape[0]

  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]

  # Modelloptimalisering - CoReg
  coreg_res, coreg_param, coreg_opt_rmse, coreg_umarkert_antall_benyttet\
  = trening_coreg_dag(rå_data,
                      treningssett_markert,
                      treningssett_umarkert,
                      treningssett,
                      testsett,
                      n_splitt,
                      n_runder,
                      random_seed=random_seed)

  # Lagring av informasjon om paramtere for treningsandel
  res_eval_opt(dict_res_opt, 'CoReg', test_andel,
               trenings_andel, coreg_param, coreg_opt_rmse, coreg_res,
               umarkert_antall, coreg_umarkert_antall_benyttet)

  # Coreg - beste modell
  # Initierer modell
  coreg = CoReg(**coreg_param)

  # Modelloptimalisering - Selv_RFR
  selv_rfr_res, selv_rfr_param, selv_rfr_opt_rmse, selv_rfr_umarkert_antall_benyttet\
  = trening_selvtrent_rfr_dag(rå_data,
                              treningssett_markert,
                              treningssett_umarkert,
                              treningssett,
                              testsett,
                              n_splitt,
                              n_runder,
                              random_seed=random_seed)

  # Lagring av informasjon om paramtere for treningsandel
  res_eval_opt(dict_res_opt, 'Selv_RFR', test_andel,
               trenings_andel, selv_rfr_param, selv_rfr_opt_rmse, selv_rfr_res,
               umarkert_antall, selv_rfr_umarkert_antall_benyttet)

  # Selv_RFR - beste modell
  # Initierer modell
  selv_rfr = SelfTrainingRandomForestRegressor(**selv_rfr_param)


  # Modelloptimalisering - BHD
  bhd_domene_res, bhd_domene_param, bhd_domene_opt_rmse, bhd_domene_umarkert_antall_benyttet\
  = trening_bhd_domene_dag(rå_data,
                            treningssett_markert,
                            treningssett_umarkert,
                            testsett,
                            n_splitt,
                            n_runder,
                            random_seed=random_seed)

  # Lagring av informasjon om paramtere for treningsandel
  res_eval_opt(dict_res_opt, 'BHD - Domene', test_andel,
               trenings_andel, bhd_domene_param, bhd_domene_opt_rmse, bhd_domene_res,
               umarkert_antall, bhd_domene_umarkert_antall_benyttet)

  # BHD - beste modell
  # Initierer modell
  bhd_domene = BHD(**bhd_domene_param)

  # Lagring av resultater fra kryssvalidering
  coreg_res = Resultat_evaluering()
  selv_rfr_res = Resultat_evaluering()
  bhd_domene_res = Resultat_evaluering()


  # Kryssvalidering
  trenings_sett, val_sett\
  = trening_validering_oppdeling_dag(rå_data,
                                     treningssett_markert,
                                     n_splitt=n_splitt,
                                     test_ekskluder_kalkun_kylling=True)

  for i, j in zip(trenings_sett, val_sett):
      train = trenings_sett[i]
      val = val_sett[j]

      # Markerte folds
      trenings_fold_markert = treningssett_markert.loc[train, :]
      val_fold = treningssett_markert.loc[val, :]

      # Markert respons
      y_trening_fold = trenings_fold_markert.iloc[:, -1]
      y_val_fold = val_fold.iloc[:, -1]


      #Fullstendig treningsdata, markert og umarkert folds
      X_trening_fold_markert = trenings_fold_markert.iloc[:, :-1]
      X_trening_fold = pd.concat([X_trening_fold_markert, X_trening_umarkert],
                                 axis=0)

      # Skalerer folds etter treningsset fold, inkl. markert og umarkert
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      # Markert
      X_trening_fold_markert_transformert = X_trening_fold_markert.copy()
      X_trening_fold_markert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_fold_markert)

      # Umarkert
      X_trening_fold_umarkert_transformert = X_trening_umarkert.copy()
      X_trening_fold_umarkert_transformert.iloc[:, :] = skalerer_fold.transform(X_trening_umarkert)

      # Validering
      X_val_fold = val_fold.iloc[:, :-1]
      X_val_fold_transformert = X_val_fold.copy()
      X_val_fold_transformert.iloc[:, :] = skalerer_fold.transform(X_val_fold)

      # Test
      X_test_transformert = X_test.copy()
      X_test_transformert.iloc[:, :] = skalerer_fold.transform(X_test)


      # CoReg
      coreg.fit(X_trening_fold_markert_transformert.values,
                y_trening_fold.values,
                X_trening_fold_umarkert_transformert.values)

      #Prediksjoner
      y_trening_pred = coreg.predict(X_trening_fold_markert_transformert.values)
      y_val_pred = coreg.predict(X_val_fold_transformert.values)
      y_test_pred = coreg.predict(X_test_transformert.values)

      # Evaluering
      coreg_res.evaluering(y_trening_fold,
                           y_trening_pred,
                           y_val_fold,
                           y_val_pred,
                           y_test,
                           y_test_pred)

      #Coreg umarkert benyttet
      coreg_umarkert_antall_benyttet = X_trening_umarkert.shape[0]

      # Selv_RFR
      selv_rfr.fit(X_trening_fold_markert_transformert.values,
                   y_trening_fold.values,
                   X_trening_fold_umarkert_transformert.values)


      # Prediksjoner
      y_trening_pred = selv_rfr.predict(X_trening_fold_markert_transformert.values)
      y_val_pred = selv_rfr.predict(X_val_fold_transformert.values)
      y_test_pred = selv_rfr.predict(X_test_transformert.values)

      # Evaluering
      selv_rfr_res.evaluering(y_trening_fold,
                              y_trening_pred,
                              y_val_fold,
                              y_val_pred,
                              y_test,
                              y_test_pred)

      #Selv_RFR umarkert benyttet
      selv_rfr_umarkert_antall_benyttet = X_trening_umarkert.shape[0] - selv_rfr.X_unlabeled.shape[0]


      # BHD -Domene

      # Prediksjoner
      y_val_pred, y_trening_pred\
      = bhd_domene.fit_predict(rå_data,
                        trenings_fold_markert,
                        treningssett_umarkert,
                        val_fold,
                        X_trening_fold_markert_transformert,
                        X_trening_fold_umarkert_transformert,
                        X_val_fold_transformert)

      y_test_pred, y_trening_pred\
      = bhd_domene.fit_predict(rå_data,
                        trenings_fold_markert,
                        treningssett_umarkert,
                        testsett,
                        X_trening_fold_markert_transformert,
                        X_trening_fold_umarkert_transformert,
                        X_test_transformert)

      # Evaluering
      bhd_domene_res.evaluering(y_trening_fold,
                         y_trening_pred,
                         y_val_fold,
                         y_val_pred,
                         y_test,
                         y_test_pred)

  # Lagring av informasjon om paramtere for

  res_eval(dict_res, 'Coreg', test_andel,
           trenings_andel, trening_markert_antall,
           parametere=coreg_param,
           rmse_score_opt=coreg_opt_rmse,
           n_splitt=n_splitt,
           res_eval=coreg_res,
           umarkert_antall=umarkert_antall,
           umarkert_benyttet=coreg_umarkert_antall_benyttet)


  res_eval(dict_res, 'Selv_RFR', test_andel,
           trenings_andel, trening_markert_antall,
           parametere=selv_rfr_param,
           rmse_score_opt=selv_rfr_opt_rmse,
           n_splitt=n_splitt,
           res_eval=selv_rfr_res,
           umarkert_antall=umarkert_antall,
           umarkert_benyttet=selv_rfr_umarkert_antall_benyttet)


  res_eval(dict_res, 'BHD - Domene', test_andel,
          trenings_andel, trening_markert_antall,
          parametere=bhd_domene_param,
          rmse_score_opt=bhd_domene_opt_rmse,
          n_splitt=n_splitt,
          res_eval=bhd_domene_res,
          umarkert_antall=umarkert_antall,
          umarkert_benyttet=bhd_domene_umarkert_antall_benyttet)